### vision RAG using [VARAG](https://github.com/adithya-s-k/VARAG)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adithya-s-k/CompanionLLadithya-s-k/VARAG/blob/main/docs/visionRAG.ipynb)

Requirement to RUN this notebook - Min T4 GPU